In [1]:
import rdflib
from rdflib import Graph, BNode, Namespace, URIRef
#from rdflib.namespace import RDF
url = 'http://dbpedia.org/resource/Donald_Trump' # Note: 'resource', not 'page'

g = rdflib.Graph()
g.load(url)
donald = next(g.subjects())

__1__) Find Donald Trump's birth place within the graph object 

[RTFM](https://rdflib.readthedocs.org/en/stable/intro_to_graphs.html)

In [2]:
#Should property or ontology be used here?
list(g.objects(predicate=URIRef('http://dbpedia.org/property/birthPlace')))

list(g.objects(predicate=URIRef('http://dbpedia.org/ontology/birthPlace')))

[rdflib.term.URIRef('http://dbpedia.org/resource/Queens'),
 rdflib.term.URIRef('http://dbpedia.org/resource/New_York')]

__2__) Add "Fred_Trump" as "relation of" to the graph object

[RTFM](https://rdflib.readthedocs.org/en/stable/intro_to_creating_rdf.html)  
[Source](https://en.wikipedia.org/wiki/Donald_Trump)

__HINT__: Check out http://dbpedia.org/ontology/

In [3]:
#Is this correct?
ns = Namespace("http://dbpedia.org/ontology/")
fred_trump = BNode()
g.add((donald, ns['relation'], fred_trump))
next(g.subject_predicates(object=fred_trump))

(rdflib.term.URIRef('http://dbpedia.org/resource/Donald_Trump'),
 rdflib.term.URIRef('http://dbpedia.org/ontology/relation'))

In [4]:
#Or is this correct?
relations = URIRef('http://dbpedia.org/property/relations')
g.add((donald, relations, rdflib.term.Literal(u'Fred_Trump')))
for relation in g.objects(donald, relations):
    print(relation)

Fred_Trump


Brian: Either is fine. It is ill specificated. The goal was to play with it in general`

---
THERE MUST BE A BETTER WAY!
---

SPARQL is SQL for RDF graphs.

SQL is a query langauge for tabular data (Pandas or databases)

SPARQL attempts to match patterns in the graph

In [6]:
try:
    from SPARQLWrapper import SPARQLWrapper, JSON
except ImportError:
    import pip
    pip.main(["install", "SPARQLWrapper"])
    from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?pred ?object
    WHERE { <http://dbpedia.org/resource/Donald_Trump> ?pred ?object}
""")
sparql.setReturnFormat(JSON)
sparql.query().convert()

{'head': {'link': [], 'vars': ['pred', 'object']},
 'results': {'bindings': [{'object': {'type': 'uri',
     'value': 'http://dbpedia.org/class/yago/Administrator109770949'},
    'pred': {'type': 'uri',
     'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}},
   {'object': {'type': 'uri',
     'value': 'http://dbpedia.org/class/yago/Adult109605289'},
    'pred': {'type': 'uri',
     'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}},
   {'object': {'type': 'uri',
     'value': 'http://dbpedia.org/class/yago/Alumnus109786338'},
    'pred': {'type': 'uri',
     'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}},
   {'object': {'type': 'uri',
     'value': 'http://dbpedia.org/class/yago/AmericanBillionaires'},
    'pred': {'type': 'uri',
     'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}},
   {'object': {'type': 'uri',
     'value': 'http://dbpedia.org/class/yago/AmericanBusinessWriters'},
    'pred': {'type': 'uri',
     'value': 'http://www.w3

__3__) Find Donald Trump's birth place within the query results

[Help](https://www.w3.org/2009/Talks/0615-qbe/)

__HINTS__: Write your query as regular SQL then rewrite in SPARQL

In [7]:
sparql.setQuery("""
    SELECT ?object
    WHERE { <http://dbpedia.org/resource/Donald_Trump> <http://dbpedia.org/ontology/birthPlace> ?object}
""")
sparql.setReturnFormat(JSON)
result = sparql.query().convert()
for i in result['results']['bindings']:
    print(i['object']['value'])

http://dbpedia.org/resource/New_York
http://dbpedia.org/resource/Queens


__4__) Using sparql, walk the graph to find a list of all the people that are also born where Donald Trump was born.

It is okay for it take multiple steps.

Brian: What about New_York and concating the results?

In [8]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?subject
    WHERE {?subject <http://dbpedia.org/ontology/birthPlace> <http://dbpedia.org/resource/Queens>}
""")
sparql.setReturnFormat(JSON)
people = sparql.query().convert()
people

{'head': {'link': [], 'vars': ['subject']},
 'results': {'bindings': [{'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Dai_Burger'}},
   {'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Miss_Ko'}},
   {'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Jasdeep_Singh'}},
   {'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Dean_Marlowe'}},
   {'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Leslie_Odom,_Jr.'}},
   {'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Priscilla_Frederick'}},
   {'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Will_Ferrara'}},
   {'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/Donald_Trump'}},
   {'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/James_Marshall_(actor)'}},
   {'subject': {'type': 'uri',
     'value': 'http://dbpedia.org/resource/John_Frusciante'}},
   {'

---
Optional
----

Parse the abstract with str methods and find as many associated elements in the RDF graph.

In [ ]:
#Todo

Which ones were easy?

Which ones were hard?

How would you do the inverse, go from a RDF to a narrative summary?

In [ ]:
#Todo